In [2]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd
import pyspark.sql.functions as psf
from math import sqrt

In [4]:
# Build a SparkSession; SparkSession provides a single
spark = SparkSession\
        .builder\
        .appName("graph")\
        .getOrCreate()

In [5]:
df = spark.read.option("header", True).csv("Speed Dating Data.csv")
df = df.select("iid","pid","match","wave")
df = df.toPandas()

result = df[df.isnull().values==True]
print(result)

df = df.drop(df.index[[1756,1766,1776,1786,1796,1806,1816,1826,1836,1846]])

result = df[df.isnull().values==True]
print(result)

      iid   pid match wave
1756  122  None     0    5
1766  123  None     0    5
1776  124  None     0    5
1786  125  None     0    5
1796  126  None     0    5
1806  127  None     0    5
1816  128  None     0    5
1826  129  None     0    5
1836  130  None     0    5
1846  131  None     0    5
Empty DataFrame
Columns: [iid, pid, match, wave]
Index: []


In [6]:
df.head(10)

,iid,pid,match,wave
0,1,11,0,1
1,1,12,0,1
2,1,13,1,1
3,1,14,1,1
4,1,15,1,1
5,1,16,0,1
6,1,17,0,1
7,1,18,0,1
8,1,19,1,1
9,1,20,0,1


In [8]:

df["iid"] = df['iid'].astype(int)
df["pid"] = df['pid'].astype(int)
df["iid"] = df["iid"] - 1
df["pid"] = df["pid"] - 1

In [9]:
df.head(10)

,iid,pid,match,wave
0,0,10,0,1
1,0,11,0,1
2,0,12,1,1
3,0,13,1,1
4,0,14,1,1
5,0,15,0,1
6,0,16,0,1
7,0,17,0,1
8,0,18,1,1
9,0,19,0,1


In [10]:
sc = spark.sparkContext
sqlContest = SQLContext(sc)
df = sqlContest.createDataFrame(df)

In [11]:
df1 = df[df["wave"] == 1]

In [12]:
dataset1 = df1.toPandas()
dataset1.head(500)

,iid,pid,match,wave
0,0,10,0,1
1,0,11,0,1
2,0,12,1,1
3,0,13,1,1
4,0,14,1,1
5,0,15,0,1
6,0,16,0,1
7,0,17,0,1
8,0,18,1,1
9,0,19,0,1


In [13]:
node = df1.select("iid","wave")
node = node.toPandas()
node = node.drop_duplicates("iid", 'first')
node = node.sort_values(by='iid')
node

,iid,wave
0,0,1
10,1,1
20,2,1
30,3,1
40,4,1
50,5,1
60,6,1
70,7,1
80,8,1
90,9,1


In [14]:
nodefile="/home/mf3200/Bigdataproject/node0(wave1).csv"
node.to_csv(path_or_buf=nodefile, header = False, sep=",")

In [15]:
nodefile = spark.read.csv("/home/mf3200/Bigdataproject/node0(wave1).csv")
node  = nodefile.toPandas()
node

,_c0,_c1,_c2
0,0,0,1
1,10,1,1
2,20,2,1
3,30,3,1
4,40,4,1
5,50,5,1
6,60,6,1
7,70,7,1
8,80,8,1
9,90,9,1


In [16]:
nodefile="/home/mf3200/Bigdataproject/node(wave1).csv"
node.to_csv(path_or_buf=nodefile, header = False, sep=",")

In [17]:
edge = df1.select("iid","pid","match","wave")
edge = edge.toPandas()
edge

,iid,pid,match,wave
0,0,10,0,1
1,0,11,0,1
2,0,12,1,1
3,0,13,1,1
4,0,14,1,1
5,0,15,0,1
6,0,16,0,1
7,0,17,0,1
8,0,18,1,1
9,0,19,0,1


In [18]:
edgefile="/home/mf3200/Bigdataproject/edge0(wave1).csv"
edge.to_csv(path_or_buf=edgefile, header = False, sep=",")

In [19]:
edgefile = spark.read.csv("/home/mf3200/Bigdataproject/edge0(wave1).csv")
edge  = edgefile.toPandas()
edge

,_c0,_c1,_c2,_c3,_c4
0,0,0,10,0,1
1,1,0,11,0,1
2,2,0,12,1,1
3,3,0,13,1,1
4,4,0,14,1,1
5,5,0,15,0,1
6,6,0,16,0,1
7,7,0,17,0,1
8,8,0,18,1,1
9,9,0,19,0,1


In [20]:
edgefile="/home/mf3200/Bigdataproject/edge(wave1).csv"
edge.to_csv(path_or_buf=edgefile, header = False, sep=",")